In [ ]:

import json
import random
import pandas as pd

def get_random_pieces(file_path, num_pieces):
    random_pieces = []
    csv_data = pd.read_csv(file_path)
    for index, row in csv_data.iterrows():
        random_pieces.append([row['text'],row['cluster_labels']])
    
    random.shuffle(random_pieces)
    return random_pieces[:num_pieces]

file_path = "annotations_medical_specialist_pre_processed_no_short.csv"
num_pieces = 5

random_pieces = get_random_pieces(file_path, num_pieces)
print(random_pieces)


In [ ]:
import json
import jsonlines
import random
import pandas as pd
import ast


def get_all_data(file_path):
    with jsonlines.open(file_path) as reader:
        data = [line for line in reader]
    return data

def get_random_pieces(file_path, num_pieces):
    random_pieces = []
    
    with jsonlines.open(file_path) as reader:
        for line in reader:
            if len(line['cluster_labels']) > 0:
                line.pop('doc_id', None)
                for annotation in line['cluster_labels']:
                    annotation[1:-1] = []
                random_pieces.append(line)
    
    random.shuffle(random_pieces)
    return random_pieces[:num_pieces]

def count_annotations(file_path, num_objects):
    
    
    top_objects = []

    with jsonlines.open(file_path) as reader:
        for line in reader:
            category_count = {
                'pathophysiology': 0, 
                'etiology': 0, 
                'epidemiology': 0, 
                'history': 0, 
                'physical': 0, 
                'exams': 0, 
                'differential': 0, 
                'therapeutic': 0
                }
            if len(line['cluster_labels']) > 0:
                for annotation in line['cluster_labels']:
                    annotation[1:-1] = []
                
                for annotation in line['cluster_labels']:
                    for item in annotation[1]:
                        if item in category_count.keys():
                            category_count[item] += 1
                count = sum(category_count[item] for item in annotation[1] if item in category_count.keys())
                categories_utilized = sum(1 for value in category_count.values() if value != 0)
                line['diversity'] = (categories_utilized/len(category_count))*count
                top_objects.append(line)

    category_count = {
        'pathophysiology': 0, 
        'etiology': 0, 
        'epidemiology': 0, 
        'history': 0, 
        'physical': 0, 
        'exams': 0, 
        'differential': 0, 
        'therapeutic': 0
        }
    
    top_objects.sort(key=lambda x: (x.get('diversity', 0)), reverse=True)
    top_objects = top_objects[:num_objects]
    
    return top_objects

def count_annotations_cluster(annotation_list,num_objects):
    top_objects = []
    for line in annotation_list.iterrows():
        category_count = {
            'pathophysiology': 0, 
            'etiology': 0, 
            'epidemiology': 0, 
            'history': 0, 
            'physical': 0, 
            'exams': 0, 
            'differential': 0, 
            'therapeutic': 0
            }

        for item_ann in line:
            if isinstance(item_ann, pd.Series):
                item_ann_dict = item_ann.to_dict()
                item_ann_dict['doc_id'] = line[0] if len(line) == 2 else None
            # annotation[1:-1] = []

        for annotation in ast.literal_eval(item_ann_dict['cluster_labels']):
            for item in annotation[1]:
                if item in category_count.keys():
                    category_count[item] += 1
        count = sum(category_count[item] for item in annotation[1] if item in category_count.keys())
        categories_utilized = sum(1 for value in category_count.values() if value != 0)
        item_ann_dict['diversity'] = (categories_utilized/len(category_count))*count
        # print(item_ann_dict['doc_id'] )
        # print(item_ann_dict['diversity'] )
        top_objects.append(item_ann_dict)
    top_objects.sort(key=lambda x: (x.get('diversity', 0)), reverse=True)
    category_count = {
            'pathophysiology': 0, 
            'etiology': 0, 
            'epidemiology': 0, 
            'history': 0, 
            'physical': 0, 
            'exams': 0, 
            'differential': 0, 
            'therapeutic': 0
            }
    return top_objects[:num_objects]
def get_top_objects(file_path, num_objects):
    objects = []
    with jsonlines.open(file_path) as reader:
        for line in reader:
            if len(line['labels']) > 0:
                for annotation in line['labels']:
                    annotation[1:-1] = []
                objects.append(line)
    
            

    objects.sort(key=lambda x: x.get('annotation', 0), reverse=True)
    top_objects = objects[:num_objects]
    
    return top_objects

In [ ]:
import pandas as pd
from datetime import datetime
file_path = "annotations_medical_specialist_pre_processed_no_short.csv"
num_objects = 11
df_annotations = pd.read_csv(file_path)
df_annotations.set_index('doc_id', inplace=True)
# print(df_annotations['cluster_labels'])

top_objects = count_annotations_cluster(df_annotations, num_objects)

# category_translation = {'pathophysiology': 'fisiopatologia', 
#                         'etiology': 'etiologia', 
#                         'epidemiology': 'epidemiologia', 
#                         'history': 'história', 
#                         'physical': 'exame físico', 
#                         'exams': 'exames complementares', 
#                         'differential': 'diagnóstico diferencial', 
#                         'therapeutic': 'tratamento',
#                         'simple':'simples',
#                         'wrong':'errado',
#                         'right':'certo',
#                         'jargon':'jargão',
#                         'encapsulated':'encapsulado'
#                         }

category_minimization = {'pathophysiology': 1, 
                        'etiology': 2, 
                        'epidemiology': 3, 
                        'history': 4, 
                        'physical': 5, 
                        'exams': 6, 
                        'differential': 7, 
                        'therapeutic': 8,
                        }
translated_top_objects = top_objects

for obj in translated_top_objects:
    category_count = {
        'pathophysiology': 0, 
        'etiology': 0, 
        'epidemiology': 0, 
        'history': 0, 
        'physical': 0, 
        'exams': 0, 
        'differential': 0, 
        'therapeutic': 0,
    }
    annotations = obj.get('cluster_labels', [])
    annotations = ast.literal_eval(annotations)
    for i in range(len(annotations)):
        # print(annotations[i])
        replacements = []
        for item in annotations[i][1]:
            if item in category_count.keys():
                category_count[item] += 1
        #         replacements.append(category_minimization[category_count[item]])
        #     else:
        #         replacements.append(item)
        
        # annotations[i][1] = replacements
    # print(category_count)
    obj['cluster_labels'] = annotations
# print(translated_top_objects)

# Convert top_objects to JSON lines format
json_lines = [json.dumps(obj,ensure_ascii=False) for obj in translated_top_objects]
json_lines_str = '\n'.join(json_lines)


now = datetime.now()

with jsonlines.open(f'prompt-shots-{now.day}-{now.month}.jsonl', 'w') as writer:
    writer.write_all(translated_top_objects)
# print(translated_top_objects)
# print(decoded_json_lines_str)
# print(translated_top_objects)
# print(str({'text':json.loads(json_lines[0])['text']}))

In [ ]:
prompt_organization_lvl = {'less':[],'semi':[],'super':[],'one':[],'two':[],'three':[],'four':[],'five':[],'six':[],'seven':[],'eight':[],'nine':[],'ten':[]}
prompt_global_score = {'less':[],'semi':[],'super':[],'one':[],'two':[],'three':[],'four':[],'five':[],'six':[],'seven':[],'eight':[],'nine':[],'ten':[]}
prompt_ideas = {'less':[],'semi':[],'super':[],'one':[],'two':[],'three':[],'four':[],'five':[],'six':[],'seven':[],'eight':[],'nine':[],'ten':[]}
prompt_ideas_full = {'less':['Preciso que você avalie o seguinte texto de um aluno de medicina. Calcule a quantidade de ideias escritas no texto. Após calcular a quantidade, retorne as partes do texto consideradas como ideias. Para responder, siga o formato em JSON a seguir:\n<<{‘text’:{texto avaliado},’annotations’:{[[‘ideia1’],[‘ideia2’],[‘ideiaN’]]}}>>\nA pergunta sobre o assunto está a seguir, começando após "P.". A resposta à pergunta começa logo após "R.".\nP. Fale TUDO que você sabe sobre infarto agudo do miocárdio. TODAS as informações que você souber são importantes.\nR.'],
                     'semi':['Preciso que você avalie o seguinte texto de um aluno de medicina. O texto pode ser dividido em unidades de ideias. Essas unidades de ideias podem ser referente a uma ou mais das categorias a seguir: fisiopatologia, epidemiologia, etiologia, história, exame físico, exames complementares, diagnóstico diferencial, tratamento.\nApós dividir o texto em unidades de ideias, retorne as partes do texto consideradas como ideias no formato a seguir:\n<<{‘text’:”texto avaliado”,’annotations’:[[‘ideia1’,[‘categoria1’,’categoriaN’]],[‘ideia2’,[‘categoria1’,’categoriaN’]],[‘ideiaN’,[‘categoria1’,’categoriaN’]]]}>>\nA pergunta sobre o assunto está a seguir, começando após "P.". A resposta à pergunta começa logo após "R.".\nP. Fale TUDO que você sabe sobre infarto agudo do miocárdio. TODAS as informações que você souber são importantes.\nR.'],
                     'super':[],
                     'one-few':['Preciso que você avalie o seguinte texto de um aluno de medicina. O texto pode ser dividido em unidades de ideias. Essas unidades podem ser referente a fisiopatologia, epidemiologia, etiologia, história, exame físico, exames complementares, diagnóstico diferencial, tratamento (algumas ideias pertencem a mais de uma dessas categorias).\nSepare e classifique as ideias no texto.\nA pergunta sobre o assunto está a seguir, começando após "P.". A resposta à pergunta começa logo após "R.".\n{{example}}\n'],
                     }


# str({'text':json.loads(json_lines[i])['text']})
# str({'annotations':json.loads(json_lines[i])['annotations']})

 #json_lines[i]

example = count_annotations(file_path, 10)
llama_config = {"top_p": 0.6,
    "temperature": 0.9,
    "top_k": 50,
    "max_new_tokens": 4096,
    "repetition_penalty": 1.03
    }

Code for Llama predict

In [ ]:
from typing import Dict, List


def format_messages(messages: List[Dict[str, str]]) -> List[str]:
    """Format messages for Llama-2 chat models.
    
    The model only supports 'system', 'user' and 'assistant' roles, starting with 'system', then 'user' and 
    alternating (u/a/u/a/u...). The last message must be from 'user'.
    """
    prompt: List[str] = []

    if messages[0]["role"] == "system":
        content = "".join(["<<SYS>>\n", messages[0]["content"], "\n<</SYS>>\n\n", messages[1]["content"]])
        messages = [{"role": messages[1]["role"], "content": content}] + messages[2:]

    for user, answer in zip(messages[::2], messages[1::2]):
        prompt.extend(["<s>", "[INST] ", (user["content"]).strip(), " [/INST] ", (answer["content"]).strip(), "</s>"])

    prompt.extend(["<s>", "[INST] ", (messages[-1]["content"]).strip(), " [/INST] "])

    return "".join(prompt)

def get_text_example(text):
    return str({"text":text})

def set_prompt(prompt_type, prompt_guide_level, prompt_pos, prompt_shot_amount):
  # user_prompt = prompt_type[prompt_guide_level][prompt_pos]

  ideas_shot_template = "P. Fale TUDO que você sabe sobre doença pulmonar obstrutiva crônica. TODAS as informações que você souber são importantes.\nR.<<{{studentResponse}}>>\nSeparação e classificação de ideias:\n<<{{annotationsResponse}}>>"
  system_prompt = "Você é um médico especialista com conhecimento médico e clínico sobre medicina interna, principalmente sobre as doenças doença pulmonar obstrutiva crônica (DPOC) e infarto agudo do miocárdio (IAM). Além disso, você consegue diferenciar entre textos de especialistas sobre IAM e DPOC e textos de alunos. Seu objetivo é avaliar textos de alunos de medicina. Não crie informações que não sejam verdadeiras. Responda no formato mostrado pelo usuário."
  
  if prompt_guide_level == 'one-few':
    example_shots = ''
    for i in range(prompt_shot_amount):
      example_shots.join(ideas_shot_template.replace('{{studentResponse}}', get_text_example(example[i]['text'])).replace('{{annotationsResponse}}', example[i]))
      user_prompt = prompt_type[prompt_guide_level][prompt_pos].replace('{{example}}', example_shots)
      user_prompt = user_prompt.replace('\\n', '\n')
  else:
    example_shots = ideas_shot_template.replace('{{studentResponse}}', get_text_example(example[0]['text'])).replace('Separação e classificação de ideias:\n<<{{annotationsResponse}}>>', '')
    user_prompt = prompt_type[prompt_guide_level][prompt_pos].replace('{{example}}', example_shots)
    user_prompt = user_prompt.replace('\\n', '\n')

  dialog = [
    { "role": "system","content": system_prompt},
    {"role": "user", "content": user_prompt}
  ]
  input = format_messages(dialog)
  return input

  
def prompt_routine(input, num_replicas, top_p,temp,top_k):
    prompt_list = []
    llama_config["top_p"] = top_p
    llama_config["temperature"] = temp 
    llama_config["top_k"] = top_k
    payload = {
    "inputs":  input,
      "parameters": {
        "do_sample": True,
        "top_p": llama_config["top_p"],
        "temperature": llama_config["temperature"],
        "top_k": llama_config["top_k"]
      }
    }
    for i in range(num_replicas):
        predictior_output = predictor.predict(payload)
        prompt_list.append(predictior_output[0]["generated_text"][len(input):])
    return prompt_list

# input = format_messages(dialog)

# payload = {
#   "inputs":  input,
#   "parameters": {
#     "do_sample": True,
#     "top_p": llama_config["top_p"],
#     "temperature": llama_config["temperature"],
#     "top_k": llama_config["top_k"],
#     # "repetition_penalty": llama_config["repetition_penalty"],
#     # "stop": ["</s>"]
#   }
# }

# send request to endpoint
# response = predictor.predict(payload)

# print(response[0]["generated_text"][len(input):])
# chat = predictor.predict({"inputs":prompt})



In [ ]:
import re

test_str = '{\n"annotations": [\n"ideia1": ["limitação do fluxo de ar na árvore respiratória"],\n"ideia2": ["resposta inflamatória crônica a agressões ambientais"],\n"ideia3": ["diagnóstico por meio de anamnese, exame físico e espirometria"],\n"ideia4": ["tratamento com broncodilatadores, corticosteroides, oxigenoterapia e antibióticos"]\n]\n}'
# Extract the values of ideias for each answer using regular expressions
ideas = re.findall(r'"ideia\d+": \["(.*?)"\]', test_str)
dict_ideas = {}
# Print the array of ideias
# print(ideas)

with open("test.txt", "r") as file:
    content = file.read()
    array = [line.replace('\n', '') for line in content.splitlines()]
    
    blocks = []
    for line in array:
        line = line.replace('\n', '')
        match = re.search(r'{(.*?)}', line)
        if match:
            block = match.group(0)
            blocks.append(block)
            # print(block)
i = 0
for block in blocks:
    block = block.replace('\n','')
    idea = re.findall(r'"ideia\d+": \["(.*?)"\]', block)
    dict_ideas[i] = idea
    i += 1
# splited_answers = blocks[0].split('],')
printed_lengths = set()
length_count = {}

for obj in dict_ideas.values():
    length = len(obj)
    if length not in printed_lengths:
        # print(length)
        printed_lengths.add(length)
    length_count[length] = length_count.get(length, 0) + 1

print("Amount of objects with each length:")
for length, count in length_count.items():
    print(f"Length {length}: {count} objects")
top_length = max(length_count, key=lambda x: x)
print("Biggest length:", top_length)


# Print the most used values throughout all objects in descending order
value_count = {}
for obj in dict_ideas.values():
    for value in obj:
        value_count[value] = value_count.get(value, 0) + 1
sorted_values = sorted(value_count.items(), key=lambda x: x[1], reverse=True)
print("Most used values:")
topN = sorted_values[:top_length]
for value, count in topN:
    print(f"{value}: {count} times")





In [ ]:
with open("test.txt", "r") as file:
    content = file.read()
    array = [line.replace('\n', '') for line in content.splitlines()]
print(array[0])

In [ ]:
import json

data = {
    "name": "John",
    "age": 30,
    "city": "New York"
}

file_path = "data.json"

with open(file_path, "w") as file:
    json.dump(data, file)


In [ ]:
import os

output_batch = get_all_data(file_path)
prompt_category = 'ideas'
guided_lvl = 'less'

report_path = f'llama-outputs/temp-{str(llama_config["temperature"])}/top-p-{str(llama_config["top_p"])}'
if not os.path.exists(report_path):
    os.makedirs(report_path)
with jsonlines.open(f'{report_path}/{prompt_category}-{guided_lvl}.jsonl', 'w') as writer:
    writer.write_all(output_batch)

In [ ]:

random_item = random.choice(output_batch)
print(random_item)
